# Introduction

- ChatGPT (Chat Generative Pre-trained Transformer) is an AI-powered chatbot created by OpenAI that enables users to have highly sophisticated, human-like conversations. The language model is capable of answering questions and assist in various tasks, including writing emails, essays, and code. Due to its dialogue design, ChatGPT is capable of answering follow-up questions, acknowledging errors, questioning incorrect assumptions, and declining inappropriate requests.

# Gradio

- Gradio is an open-source Python library that is used to build machine learning and data science demos and web applications.